# 0-1 Knapsack example

## Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# optimization
from optimization import QuadraticProgram
from optimization.workflows import QUBO_transformer
from optimization.passes import (QUBO2Ising,
                                 EvaluateProgramSolution,
                                 UnrollQUBOVariables)

# Docplex
from docplex.mp.model import Model

# SPSA
from optimization.spsa import minimize_spsa

## Load the Runtime (if using)

In [2]:
#service = QiskitRuntimeService()

In [3]:
#backend = service.get_backend('ibm_nazca')

## Step 1: Build optimization problem using docplex and convert to QUBO
Define problem using standard classical representation.

In [4]:
values = [3, 4, 5, 6, 7]
weight_of_items = [2, 3, 4, 5, 6]
max_weight = 12

In [5]:
mdl = Model(name="Knapsack")
x = {i: mdl.binary_var(name=f"x_{i}") for i in range(len(values))}
mdl.maximize(mdl.sum(values[i] * x[i] for i in x))
mdl.add_constraint(mdl.sum(weight_of_items[i] * x[i] for i in x) <= max_weight)

docplex.mp.LinearConstraint[](2x_0+3x_1+4x_2+5x_3+6x_4,LE,12)

### Convert to our `QuadraticProgram` format

In [6]:
qp = QuadraticProgram.from_docplex_mp(mdl)
qp

<QuadraticProgram: maximize 3*x_0 + 4*x_1 + 5*x_2 + 6*x_3 + 7*x_4, 5 variables, 1 constraints, 'Knapsack'>

### Perform transformations to QUBO and return new program

In [7]:
qubo_transformer = QUBO_transformer()
qubo = qubo_transformer.run(qp)

## Step 2) Quantum native description
### Convert from QUBO to Ising

In [8]:
hamiltonian = QUBO2Ising().run(qubo)
hamiltonian

SparsePauliOp(['IIIIIIIIZ', 'IIIIIIIZI', 'IIIIIIZII', 'IIIIIZIII', 'IIIIZIIII', 'IIIZIIIII', 'IIZIIIIII', 'IZIIIIIII', 'ZIIIIIIII', 'IIIIIIIZZ', 'IIIIIIZIZ', 'IIIIIZIIZ', 'IIIIZIIIZ', 'IIIZIIIIZ', 'IIZIIIIIZ', 'IZIIIIIIZ', 'ZIIIIIIIZ', 'IIIIIIZZI', 'IIIIIZIZI', 'IIIIZIIZI', 'IIIZIIIZI', 'IIZIIIIZI', 'IZIIIIIZI', 'ZIIIIIIZI', 'IIIIIZZII', 'IIIIZIZII', 'IIIZIIZII', 'IIZIIIZII', 'IZIIIIZII', 'ZIIIIIZII', 'IIIIZZIII', 'IIIZIZIII', 'IIZIIZIII', 'IZIIIZIII', 'ZIIIIZIII', 'IIIZZIIII', 'IIZIZIIII', 'IZIIZIIII', 'ZIIIZIIII', 'IIZZIIIII', 'IZIZIIIII', 'ZIIZIIIII', 'IZZIIIIII', 'ZIZIIIIII', 'ZZIIIIIII', 'IIIIIIIII'],
              coeffs=[-206.5+0.j, -310. +0.j, -413.5+0.j, -517. +0.j, -620.5+0.j, -104. +0.j,
 -208. +0.j, -416. +0.j, -520. +0.j,   78. +0.j,  104. +0.j,  130. +0.j,
  156. +0.j,   26. +0.j,   52. +0.j,  104. +0.j,  130. +0.j,  156. +0.j,
  195. +0.j,  234. +0.j,   39. +0.j,   78. +0.j,  156. +0.j,  195. +0.j,
  260. +0.j,  312. +0.j,   52. +0.j,  104. +0.j,  208. +0.j,  260. +0.j,


### Select ansatz circuit from circuit library

In [9]:
ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=2)

## Step 3: Solve using quantum accelerator

### Standard cost function definition

In [10]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [11]:
#session = Session(backend=backend)
#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

### Perform minimization

In [12]:
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=100)

### Computute distribution at found minimum

In [13]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

### Express quantum solution as QUBO variable values
Convert bit-string solutions to variable values in QUBO problem

In [14]:
res = EvaluateProgramSolution(qubo).run(samp_dist)
res

(-15.0, array([0, 1, 1, 1, 0, 0, 0, 0, 0]))

## Step 4: Classical postprocess
Unrolling QUBO variables back to the original problem definition, if needed

In [15]:
solution = UnrollQUBOVariables(qubo_transformer).run(res)
solution

array([0., 1., 1., 1., 0.])

## Intepretation of solution
Depends on input, but is completely separate from quantum having done the processing for the solution.

In [16]:
sum(np.array(weight_of_items)*solution)

12.0